In [3]:
import pandas as pd
import string
import random
import pickle
import numpy as np

In [4]:
with open("ngram_data/word_dict.pickle","rb") as f:
    word_dict = pickle.load(f)
with open("ngram_data/word_prob_dict.pickle","rb") as f:
    word_prob_dict = pickle.load(f)

In [5]:
word_freq=pd.read_csv("ngram_data/word_freq.csv").set_index("word")["frequency"].to_dict()

In [6]:
bigram=pd.read_csv("ngram_data/2gram.csv").set_index("2-gram")["frequency"].to_dict()
trigram=pd.read_csv("ngram_data/3gram.csv").set_index("3-gram")["frequency"].to_dict()

In [7]:
with open("ngram_data/5gram_probs.pickle", "rb") as f:
    fivegramprobs = pickle.load(f)

In [8]:
def encrypt(plaintext,key):
    trans = plaintext.maketrans(key,string.ascii_lowercase)
    cipher = plaintext.translate(trans)
    return cipher

def decrypt(ciphertext,key):
    trans = ciphertext.maketrans(string.ascii_lowercase,key)
    cipher = ciphertext.translate(trans)
    return cipher

def paternify(instr):
    ht = dict()
    pattern = ""
    cnt = 97
    for i in instr:
        if i in ht:
            pattern = pattern + ht[i]
        else:
            pattern = pattern + chr(cnt)
            ht[i]=chr(cnt)
            cnt = cnt + 1
    return pattern

def bigram_score(instring,bigram):
    prob = None
    for i,j in zip(instring, instring[1:]):
        if i+j in bigram:
            if prob==None:
                prob = np.log(bigram[i+j])
            else:
                prob=prob + np.log(bigram[i+j])
        
    return prob

def trigram_score(instring,trigram):
    prob = None
    for i,j,z in zip(instring, instring[1:],instring[2:]):
        if i+j+z in trigram:
            if prob==None:
                prob = np.log(trigram[i+j+z])
            else:
                prob=prob + np.log(trigram[i+j+z])
        
    return prob

def word_score(instring,word_freq):
    oov_score = np.log(10**(-10))
    words = instring.split(" ")
    prob = 0
    for i in words:
        if i in word_freq:
            prob += np.log(word_freq[i])
        else:
            prob += oov_score
            
    return prob

def transify(str1,str2):
    trans = dict()
    
    fxu = list(set(str1)-set(str2))
    fxl = list(set(str2)-set(str1))
    
    for i,j in zip(str1,str2):
        if ord(i) not in trans:
            trans[ord(i)]=ord(j)
    
    for i,j in zip(fxu,fxl):
        trans[ord(j)]=ord(i)

    return trans

def isconsistent(assignment,pattern, word):

    for i,j in zip(pattern,word):
        if ord(i) in assignment:
            if assignment[ord(i)]!=ord(j):
                return False
        elif ord(j) in assignment.values(): # avoids the case when letter is assigned to a different value
            return False
            
    return True

def get_assignment(word1,word2):
    trans = dict()
    for i,j in zip(word1,word2):
        if i not in trans:
            trans[ord(i)]=ord(j)
    return trans


def assignment_trans(assignment):
    
    vl = set(assignment.values())
    ks = set(assignment.keys())
    
    fxu = list(ks - vl)
    fxl = list(vl - ks)
    
    for i,j in zip(fxu,fxl):
        assignment[j]=i

    return assignment


def get_score(charseq, ngram_probs, word_frequency):
    totprob = 0
    words = charseq.split(" ")
    sz = len(list(ngram_probs.keys())[0])
    eps = np.log(1e-9)

    for i in words:
        if i in word_frequency:
            totprob += np.log(word_frequency[i])
        else:
            totprob += eps

    for i in range(len(charseq)-sz+1):
        cprob = ngram_probs.get(charseq[i:i+sz],0)
        if cprob == 0:
            totprob += eps
        else:
            totprob += np.log(cprob)

    return totprob


def get_scorev2(charseq, ngram_probs,):
    totprob = 0
    words = charseq.split(" ")
    sz = len(list(ngram_probs.keys())[0])
    eps = np.log(1e-9)

    for i in range(len(charseq)-sz+1):
        cprob = ngram_probs.get(charseq[i:i+sz],0)
        if cprob == 0:
            totprob += eps
        else:
            totprob += np.log(cprob)

    return totprob

In [9]:
actual_key = "badcfehgjilkonmrqputsxwvzy"
plaintext =  "key quick brown jumps over lazy dog the fox"
# plaintext = "sentence that definitely has a unique solution"
# plaintext = "world science technology"
cipher = encrypt(plaintext,actual_key)

In [1]:
%%time
def solver(cipher, word_dict, ngram_probs):
    key = string.ascii_lowercase
    word_seq = cipher.split(" ")
    
    solutions = []
    partial_solutions=[]
    
    que = []
    que.append((0,{}))
    while len(que)!=0:
        
        index, assignment = que.pop()
        if index >= len(word_seq):
            cor_assign = assignment_trans(assignment)
            sol = cipher.translate(cor_assign)
            solutions.append((sol, get_score(sol,ngram_probs, word_freq)))

        else:
            values = word_dict[paternify(word_seq[index])]
            was_assign = False
            for sub_word in values:
                if isconsistent(assignment,word_seq[index],sub_word):
                    cassignment = dict()
                    cassignment.update(assignment)
                    cassignment.update(get_assignment(word_seq[index],sub_word))
                    que.append((index+1,cassignment))
                    was_assign=True
            if was_assign==False:
                cor_assign = assignment_trans(assignment)
                partial_solutions.append(cipher.translate(cor_assign))
    
    return (solutions,partial_solutions)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.01 µs


In [2]:
solver(cipher,word_dict,fivegramprobs)

NameError: name 'cipher' is not defined

## END OF FILE

In [ ]:
random.seed(42)
np.random.seed(42)
def old_solve(cipher,word_dict,word_prob_dict):
    key=string.ascii_lowercase
    plaintext=decrypt(cipher,key)
    best_score = get_score(plaintext,fivegramprobs, word_freq)
    for i in range(10000):
        word_seq = plaintext.split(" ")
        len_seq = np.array([len(i) for i in word_seq])
        len_seq = len_seq/sum(len_seq)
        rnd_word = np.random.choice(word_seq,p=len_seq)
        rnd_word_pattern = paternify(rnd_word)
        word_sub_prob=word_prob_dict[rnd_word_pattern]/np.sum(word_prob_dict[rnd_word_pattern])
        sub_word = np.random.choice(word_dict[rnd_word_pattern],p=word_sub_prob)
        trans = transify(rnd_word,sub_word)
        mod_key = key.translate(trans)
        mod_plaintext=decrypt(cipher,mod_key)
        if(get_score(mod_plaintext,fivegramprobs, word_freq)>best_score):
            key=mod_key
            plaintext=mod_plaintext
            best_score=get_score(mod_plaintext,fivegramprobs, word_freq)
            print(best_score,plaintext)
        
        
        
old_solve(cipher,word_dict,word_prob_dict)

In [38]:
import pickle
with open("ngram_data/onegram.pickle","rb") as f:
    onegram = pickle.load(f)/100

In [39]:

def swap_chars(st,a,b):
    result = ""
    for i in st:
        if i == a:
            result += b
        elif i == b:
            result += a
        else:
            result +=i
    return result

In [40]:
random.seed(42)
np.random.seed(42)
def old_solve(cipher,word_dict,word_prob_dict):
    key=string.ascii_lowercase
    plaintext=decrypt(cipher,key)
    best_score = get_scorev2(plaintext,fivegramprobs)
    print(best_score)
    for i in range(10000):
        a,b=np.random.choice(list(string.ascii_lowercase),replace=False,size=2,p=onegram)
        mod_key = swap_chars(key,a,b)
        mod_plaintext=decrypt(cipher,mod_key)
        if(get_scorev2(mod_plaintext,fivegramprobs)>best_score):
            key=mod_key
            plaintext=mod_plaintext
            best_score=get_scorev2(mod_plaintext,fivegramprobs)
            print(best_score,plaintext)
        
        
        
old_solve(cipher,word_dict,word_prob_dict)

-797.7912435025544
-795.3528095410281 lfz qsjdl apmwr isonu mxfp kbyz cmh tgf emv
-792.9783493783582 lfz qijdl apmwr sionu mxfp kbyz cmh tgf emv
-773.3616063335083 lfz qijdl upmwr siona mxfp kbyz cmh tgf emv
-769.7569852776843 lfz qijdl upmwr tiona mxfp kbyz cmh sgf emv
-765.6466195925049 lfz qijdl upmwr tions mxfp kbyz cmh agf emv
-760.8044403277421 lfz qijdl upmar tions mxfp kbyz cmh wgf emv
-759.8770177967489 lfz qijdl uemar tions mxfe kbyz cmh wgf pmv
-756.1146258021201 mfz qijdm uelar tions lxfe kbyz clh wgf plv
-747.6839653593914 mfz qijdm ualer tions lxfa kbyz clh wgf plv
-737.0970320785449 mfz qijdm haler tions lxfa kbyz clu wgf plv
-730.0670406594735 mfz qijdm haver tions vxfa kbyz cvu wgf pvl


KeyboardInterrupt: 

In [45]:
# random.seed(42)
# np.random.seed(42)
def old_solve(cipher,word_dict,word_prob_dict):
    key=string.ascii_lowercase
    plaintext=decrypt(cipher,key)
    best_score = get_score(plaintext,fivegramprobs,word_freq)
    print(best_score)
    for i in range(10000):
        a,b=np.random.choice(list(string.ascii_lowercase),replace=False,size=2,p=onegram)
        mod_key = swap_chars(key,a,b)
        mod_plaintext=decrypt(cipher,mod_key)
        if(get_score(mod_plaintext,fivegramprobs,word_freq)>best_score):
            key=mod_key
            plaintext=mod_plaintext
            best_score=get_score(mod_plaintext,fivegramprobs,word_freq)
            print(best_score,plaintext)
        
        
        
old_solve(cipher,word_dict,word_prob_dict)

-984.3006360350723
-966.355145182804 lfz qsjdl apmwn istru mxfp kbyz cmh ogf emv
-959.6657328745281 lfz qsjdl apmen istru mxfp kbyz cmh ogf wmv
-954.873118468554 lyz qsjdl apmen istru mxyp kbfz cmh ogy wmv
-940.4551991613199 myz qsjdm aplen istru lxyp kbfz clh ogy wlv
-936.3557319359057 myz qsjdm aplen istru lxyp kbfz clh goy wlv
-932.2562647104917 myz qsjhm aplen istru lxyp kbfz cld goy wlv
-928.6040997696279 myz qsjhm aplen ustri lxyp kbfz cld goy wlv
-924.5241061133925 myz qsjhm palen ustri lxya kbfz cld goy wlv
-922.7323437436506 myz qsjhm palen ustri lxya kbfz dlc goy wlv
-921.6940493217874 myz qsjhm panel ustri nxya kbfz dnc goy wnv
-921.0061327619158 myz qsjhm penal ustri nxye kbfz dnc goy wnv
-919.6198340500283 myz qsjhm penal ustri nxye kbgz dnc foy wnv
-918.9952359314058 miz qsjhm penal ustry nxie kbgz dnc foi wnv
-913.5035495783399 miz qsjhm penal ustry nxie kogz dnc fbi wnv
-912.0386101920654 miz qsjhm penal ustro nxie kygz dnc fbi wnv
-908.6980259538572 miz qschm penal ust

In [10]:
# def old_solve(cipher,word_dict,word_prob_dict):
#     key=string.ascii_lowercase
#     plaintext=decrypt(cipher,key)
#     best_score = word_score(plaintext,word_freq)

#     for i in range(10000):
#         word_seq = plaintext.split(" ")
#         len_seq = np.array([len(i) for i in word_seq])
#         len_seq = len_seq/sum(len_seq)
#         rnd_word = np.random.choice(word_seq,p=len_seq)
        
#         rnd_word_pattern = paternify(rnd_word)
#         word_sub_prob=word_prob_dict[rnd_word_pattern]/np.sum(word_prob_dict[rnd_word_pattern])
        
#         sub_word = np.random.choice(word_dict[rnd_word_pattern],p=word_sub_prob)
#         trans = transify(rnd_word,sub_word)
#         mod_key = key.translate(trans)
#         mod_plaintext=decrypt(cipher,mod_key)
#         if(word_score(mod_plaintext,word_freq)>best_score):
#             key=mod_key
#             plaintext=mod_plaintext
#             best_score=word_score(mod_plaintext,word_freq)
#             print(best_score,plaintext)
        
        
        
# solve(cipher,word_dict,word_prob_dict)

-268.5777555542455 femj asdgterub bm otjaht bm other momnr omnru jtlr oraetu esn otnz ohyotyu
-264.1994917677408 fimj ksdeniagb bm tnjkhn bm tnhia mtmoa tmoag jnla taking iso tnoz thytnyg
-257.13089216442813 fcmj syndicate em bijshi em bihca mbmoa bmoat jila bascit cyo bioz bhgbigt
-251.7735704596357 fvmj discovery ym bojdho ym bohve mbmae bmaer jole bedvor via boaz bhgbogr
-243.52533624890765 fdmj vascoderi im bojvho im bohde mbmye bmyer jole bevdor day boyz bhgbogr
-237.03765230988915 fdmb vascodhri im lobveo im loedh mlmyh lmyhr both lhvdor day loyz leglogr
-236.60473063674957 vdmf oascedhli im before im berdh mbmyh bmyhl feth bhodel day beyz brgbegl
-234.67667809602315 vdfc oasredhli if become if bemdh fbfyh bfyhl ceth bhodel day beyz bmgbegl
-232.58688365866166 vwfc oayrewhli if become if bemwh fbfsh bfshl ceth bhowel was besz bmgbegl
-231.31638010566104 vafc onyreahli if become if bemah fbfdh bfdhl ceth bhoael and bedz bmgbegl
-228.89461298921572 vatc onyreahli it become it bemah

In [ ]:
import string
import numpy as np
from sympy import Matrix, init_printing
init_printing()
ciphertext="n irel ybat fragrapr gung dhvm dhvm fbyire vf hanoyr gb qrpelmg un un orpnhfr vg vf dhvgr qhzo"
# key = string.ascii_lowercase
key = "nopqrstuvwxyzabcdefghijklm"
freq_mat = generate_trig_freq_matrix(ciphertext,key)
c_score=get_score(freq_mat,trigram)
g_score=c_score
k = 0
tot=0
while True:
    k+=1
    tot+=1
    a,b=np.random.choice(list(string.ascii_lowercase),replace=False,size=2,p=onegram)
    putative_key = swap_chars(key,a,b)
    putative_freq_mat = fswap_trig(freq_mat,a,b)
    putative_score = get_score(putative_freq_mat,trigram)
    
    if k>10000:
        key_=list(key)
        random.shuffle(key_)
        key=''.join(key_)
        freq_mat = generate_trig_freq_matrix(ciphertext,key)
        c_score=get_score(freq_mat,trigram)    
        k=0
        
    if putative_score < c_score:
        key=putative_key
        freq_mat = putative_freq_mat
        c_score = putative_score
        k=0
        #print(key,decrypt_cipher(ciphertext,key))
    if putative_score < g_score:
        gkey=putative_key
        g_score=putative_score
        print(key,decrypt_cipher(ciphertext,gkey),g_score)